## Monitoring Vibration Data
Notebook to manually monitor and assess the vibration data from the IoT Connxt platform and the IFM sensors. The notebook performs the following steps:
- reading necessary packages and authenticating
- retrieving time series objects
- retrieving recent time series
- visualizing the data

In [ ]:
import os
import requests
from datetime import datetime
import pandas as pd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from dotenv import load_dotenv

Authenticate yourself to retrieve data in Lizard. For this, you need an API key from Lizard.

In [ ]:
# Get API key from env file
load_dotenv()

username = '__key__'
password = os.getenv("LIZARD_API_KEY")
json_headers = {
            "username": username,
            "password": password,
            "Content-Type": "application/json",
        }

### Retrieve time series objects from pumping station Zuidbroek

In [ ]:
# Define gemaal based on naam in Damo
gemaal = "Rg. Zuidbroek"

url = "https://demo.lizard.net/api/v4/timeseries/?location__name__startswith="+gemaal

# Get the data from the API
print(url)
print(requests.get(url,headers=json_headers,params= {'page_size':'10000'}).json())
querydata = requests.get(url,headers=json_headers,params= {'page_size':'10000'}).json()['results']
timeseries_list = pd.DataFrame(querydata)


### Retrieving time series and displaying the latest observation in a table


Retrieve the latest value for all time series objects found for pumping station Zuidbroek and present it in a table based on recency.

In [ ]:
timeseries_list['location'] = timeseries_list['location'].apply(lambda x: x['name'])
timeseries_list['last_modified'] = pd.to_datetime(timeseries_list['last_modified'])
timeseries_list = timeseries_list.sort_values(by='last_modified', ascending=False)
timeseries_list['last_modified'] = timeseries_list['last_modified'].dt.tz_convert('CET').dt.strftime('%Y-%m-%d %H:%M:%S')
timeseries_list = timeseries_list[['uuid','location', 'name', 'last_modified', 'last_value']]
timeseries_list

### Visualizing Data
Use the UUID of a time series to retrieve and display the data for this parameter from the past week in a graph.

In [ ]:
# make list of uuids for all rows in timeserieslist with sensor in the name
uuids = timeseries_list[timeseries_list['location'].str.contains('IoT sensor', case=False)]['uuid'].tolist()
uuids = timeseries_list[timeseries_list['location'].str.contains('sensor', case=False)]['uuid'].tolist()

start = (datetime.now() - pd.Timedelta(days=14)).strftime('%Y-%m-%dT%H:%M:%SZ')
end = (datetime.now() - pd.Timedelta(days=0)).strftime('%Y-%m-%dT%H:%M:%SZ')
base_url = "https://hhnk.lizard.net/api/v4/timeseries/{}/events/"

all_counts_list = []

for uuid in uuids:
    url = base_url.format(uuid)
    time_series_events = pd.DataFrame(requests.get(url=url, headers=json_headers, params={'start': start, 'end': end, 'fields': 'value', 'page_size': '10000000'}).json()['results'])
    
    # Process the time series events
    time_series_events['time'] = time_series_events['time'].str.replace(r'\.\d+Z', 'Z', regex=True) # remove milliseconds
    time_series_events['timestamp'] = pd.to_datetime(time_series_events['time'])
    time_series_events['date'] = time_series_events['timestamp'].dt.date
    time_series_events['count'] = 1
    ts_count = time_series_events.groupby('date').count()
    ts_count = ts_count[['count']]
    # Get based on uuid the location from timeseries_list and use as column name
    location = timeseries_list[timeseries_list['uuid'] == uuid]['location'].values[0]
    ts_count.columns = [location]
    
    all_counts_list.append(ts_count)

# Merge all counts into one DataFrame
all_counts_df = pd.concat(all_counts_list, axis=1)
all_counts_df = all_counts_df.iloc[1:]
all_counts_df = all_counts_df.sort_index()
all_counts_df.columns = all_counts_df.columns.str.replace('Rg. Zuidbroek ', '')
all_counts_df.columns = all_counts_df.columns.str.replace('Rg. Zuidbroek,', '')

# Show table with columns as rows and dates as columns and columns descending based on date
counts_df = all_counts_df.T
counts_df = counts_df.sort_index(ascending=True)
counts_df.index.name = 'location'
counts_df

In [ ]:
fig, axes = plt.subplots(nrows=13, ncols=2, figsize=(15, 30))
axes = axes.flatten()

for i, column in enumerate(all_counts_df.columns):
    all_counts_df[column].plot(ax=axes[i], title=column)
    axes[i].set_xlabel('Date')
    axes[i].set_ylabel('Count')
    axes[i].get_legend().remove()

plt.tight_layout()
plt.show()
